In [130]:
# Audio to text

import os
import os
from tqdm import tqdm
import whisper
import subprocess
import string

os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\bin"

def check_ffmpeg():
    try:
        cmd = ["ffmpeg", "-version"]
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            print("ffmpeg is accessible")
        else:
            print("ffmpeg is not accessible")
    except FileNotFoundError:
        print("ffmpeg not found")

check_ffmpeg()

model = whisper.load_model("base")

def transcribe_audio(file_path):
    audio = whisper.load_audio(file_path)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    _, probs = model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16=False)
    result = whisper.decode(model, mel, options)

    transcription = result.text.translate(str.maketrans('', '', string.punctuation)).lower()

    return transcription

flac_file_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\k3s2_gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00002.flac'

if os.path.isfile(flac_file_path):
    transcription = transcribe_audio(flac_file_path)
    print(f"Final Transcription: {transcription}")
else:
    print("File not found. Please check the file path.")


wav_directory = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\k3s2_gmmGroup\audioResults\VitsModel'

transcriptions = []
for filename in tqdm(os.listdir(wav_directory), desc="Processing files"):
    if filename.endswith(".flac"):
        file_path = os.path.join(wav_directory, filename)
        transcription = transcribe_audio(file_path)
        transcriptions.append(transcription)



ffmpeg is accessible
Final Transcription: security so it doesnt feel very secure to me as a farmer to hear that in my family and i have been farming here since 1990


Processing files: 100%|██████████| 335/335 [04:27<00:00,  1.25it/s]


In [131]:
# Save to txt

output_file_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\k3s2_gmmGroup\audioResults\transcriptions_VitsModel.txt'
with open(output_file_path, 'w') as f:
    for transcription in transcriptions:
        f.write(transcription + '\n')

In [132]:
# txt to List

output_file_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\k3s2_gmmGroup\audioResults\transcriptions_VitsModel.txt'
with open(output_file_path, 'r') as f:
    transcriptions_from_file = [line.strip() for line in f]

In [133]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="clean.json")

textInput = [] 
audioPathInput = [] 

for data in dataset['train']['training_data']:
    textInput.extend(data['label'])
    audioPathInput.extend(data['name'])

printInputArrays = False

if printInputArrays:
    for i in range(len(textInput)):
        print(textInput[i])

    for i in range(len(audioPathInput)):
        print(audioPathInput[i])

In [134]:
# Calculate the WER between two texts

# Objective Evaluation
# Word Error Rate (WER):

# Transcribe the generated audio back to text using an automatic speech recognition (ASR) system.
# Compare the transcribed text with the original text to calculate the WER. The model with the lower WER is better.

from jiwer import wer

if len(textInput)==len(transcriptions_from_file):
    original_texts = textInput
    transcribed_text_model1 = transcriptions_from_file
    transcribed_text_model2 = []

    wer_list_model1 = [wer(original, transcribed) for original, transcribed in zip(original_texts, transcribed_text_model1)] if transcribed_text_model1 else []
    wer_list_model2 = [wer(original, transcribed) for original, transcribed in zip(original_texts, transcribed_text_model2)] if transcribed_text_model2 else []

    average_wer_model1 = sum(wer_list_model1) / len(wer_list_model1) if wer_list_model1 else float('inf')
    average_wer_model2 = sum(wer_list_model2) / len(wer_list_model2) if wer_list_model2 else float('inf')

    print(f"Average WER Model 1: {average_wer_model1}")
    print(f"Average WER Model 2: {average_wer_model2}")
else:
    print("Input arrays are not of the same length.")
    print(f"textInput: {len(textInput)} transcriptions_from_file: {len(transcriptions_from_file)}")


Average WER Model 1: 0.12049162824542967
Average WER Model 2: inf


In [ ]:
# Perceptual Evaluation of Speech Quality (PESQ):

# Use PESQ to measure the quality of the speech signals. PESQ compares the generated audio to a reference audio signal and gives a quality score.
# Libraries like pypesq can be used for this purpose.

In [13]:
from pydub import AudioSegment
from scipy.io import wavfile
import os

def convert_flac_to_wav(flac_file_path, wav_file_path):
    audio = AudioSegment.from_file(flac_file_path, format="flac")
    audio.export(wav_file_path, format="wav")

# File paths
ref_flac_path = r'audioResults/VitsModel/07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac'
#gen_flac_path_model1 = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac'
#gen_flac_path_model2 = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00001.flac'


# File paths
ref_wav_path = r'output.wav'
#gen_wav_path_model1 = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.wav'
#gen_wav_path_model2 = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00001.wav'


convert_flac_to_wav(ref_flac_path, ref_wav_path)
#convert_flac_to_wav(gen_flac_path_model1, gen_wav_path_model1)
#convert_flac_to_wav(gen_flac_path_model2, gen_wav_path_model2)

# Read WAV files
rate, ref_audio = wavfile.read(ref_wav_path)
#rate, gen_audio_model1 = wavfile.read(gen_wav_path_model1)
#rate, gen_audio_model2 = wavfile.read(gen_wav_path_model2)

# Compute PESQ scores
pesq_score_model1 = pesq(rate, ref_audio, gen_audio_model1, 'wb')
#pesq_score_model2 = pesq(rate, ref_audio, gen_audio_model2, 'wb')

print(f"PESQ Score Model 1: {pesq_score_model1}")
#print(f"PESQ Score Model 2: {pesq_score_model2}")

# Clean up temporary WAV files if needed
os.remove(ref_wav_path)
os.remove(gen_wav_path_model1)
os.remove(gen_wav_path_model2)


CouldntDecodeError: Decoding failed. ffmpeg returned error code: 183

Output from ffmpeg/avlib:

ffmpeg version n6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.2.1 (GCC) 20230801
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enable-librsvg --enable-librubberband --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --e

In [7]:
from pypesq import pesq
from scipy.io import wavfile

rate, ref_audio = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\data\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac')
rate, gen_audio_model1 = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac')
rate, gen_audio_model2 = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00001.flac')

pesq_score_model1 = pesq(rate, ref_audio, gen_audio_model1, 'wb')
pesq_score_model2 = pesq(rate, ref_audio, gen_audio_model2, 'wb')

print(f"PESQ Score Model 1: {pesq_score_model1}")
print(f"PESQ Score Model 2: {pesq_score_model2}")


ModuleNotFoundError: No module named 'pypesq'

In [6]:
!pip install --use-pep517 pypesq

  Using cached pypesq-1.2.4.tar.gz (30 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [61 lines of output]
      running dist_info
      creating /tmp/pip-modern-metadata-pdc80b43/pypesq.egg-info
      writing /tmp/pip-modern-metadata-pdc80b43/pypesq.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-modern-metadata-pdc80b43/pypesq.egg-info/dependency_links.txt
      writing requirements to /tmp/pip-modern-metadata-pdc80b43/pypesq.egg-info/requires.txt
      writing top-level names to /tmp/pip-modern-metadata-pdc80b43/pypesq.egg-info/top_level.txt
      writing manifest file '/tmp/pip-modern-metadata-pdc80b43/pypesq.egg-info/SOURCES.txt'
      file numpy.py (for module numpy) not found
      Traceback (most recent 